In [1]:
# 引進基本資料
import pandas as pd
import csv
import numpy as np

In [2]:
# 將所有的rules跑進這裡
data = {}
for i in range(1, 6):
    filename = f"C:/Users/user/Desktop/people_base(new)/people_base_age{i}(New).csv"
    data[f"age{i}"] = pd.read_csv(filename)

for i in range(1, 4):
    filename = f"C:/Users/user/Desktop/people_base(new)/people_base_level{i}(New).csv"
    data[f"level{i}"] = pd.read_csv(filename)

for i in range(1, 6):
    for j in range(1,4):
        filename = f"C:/Users/user/Desktop/people_base(new)/people_base_age{i}_level_{j}(New).csv"
        data[f"age{i}level{j}"] = pd.read_csv(filename)    
print(len(data))

23


In [3]:
# 取得23個購買紀錄basket
all_basket = []
for i in data:
    basket = []
    for _,row in data[i].iterrows():
        try:
            buy_list_str = list(row[1].strip("}{").split(","))
            buy_list = list(map(int, buy_list_str))
            if len(buy_list) >0:
                basket.append(buy_list)
        except:
            continue
    all_basket.append(basket)
print(len(all_basket))
# all_basket: len=23, with different group

23


In [4]:
# 觀察籃子內容
print(all_basket[9])

[[42, 109], [105], [3, 5, 6, 40, 109], [114, 19, 111], [109, 20, 5], [24, 3, 112, 5], [54], [54], [34, 11, 4], [112, 1, 34, 11], [4, 109], [112, 1, 50, 43], [2, 4, 110], [1, 4, 109], [17, 2, 109, 7], [9, 74, 19, 109], [33, 4, 110], [1, 3, 109], [54], [40, 3, 112], [112, 13, 15], [80, 49, 12, 101], [115, 49, 19, 4], [109, 6], [54]]


In [5]:
new = dict()
names = list(data.keys())
r = 0
for i in data:
    new[i] = all_basket[r]
    r += 1

In [6]:
# 創造23組規則
from apyori import apriori
columns = ['buy', 'maybuy', 'support', 'confidence', 'lift']
all_rules = dict()
from tqdm import tqdm, trange
for name, basket in tqdm(new.items()):
    print(name)
    print(len(basket))
    frequent_itemsets = apriori(basket, min_support=0.015,min_confidence=0.2, min_lift=1.5, use_colnames=False)
    rules = list(frequent_itemsets)
    df = pd.DataFrame(columns=columns)
    for product in rules:
        for j in range(len(product[2])):
            if float(product[2][j][2]) > 0.9:
                continue
            data = [list(product[2][j][0]), list(product[2][j][1]), float(product[1]),float(product[2][j][2]),float(product[2][j][3])]
            df = pd.concat([df, pd.DataFrame([data], columns=df.columns)], ignore_index=True)
    all_rules[name] = df

 13%|█▎        | 3/23 [00:00<00:00, 27.59it/s]

age1
211
age2
10420
age3
21206
age4
16066
age5
11038
level1
40874


 43%|████▎     | 10/23 [00:00<00:00, 22.48it/s]

level2
10758
level3
7311
age1level1
171
age1level2
25
age1level3
13


100%|██████████| 23/23 [00:00<00:00, 29.84it/s]

age2level1
7420
age2level2
1861
age2level3
1137
age3level1
14768
age3level2
3887
age3level3
2549
age4level1
11165
age4level2
2884
age4level3
2015
age5level1
7346
age5level2
2097
age5level3
1593


In [7]:
# 處理標籤資料
item = pd.read_csv("C:/Users/user/Desktop/BDA/multihot_vector_new.csv")
print(item.head())

                SalePage_Id  \
0  jq7lb1I7BO5fTOdpb+5HIg==   
1  /A2P26xd+Fdx5fqwlBHklQ==   
2  n86xSkA4ACr3tQy4qC/qHg==   
3  XEeHbHVhPtbIyUn7/EO/JQ==   
4  i86M+aBbkl6WhQUOhXe4TQ==   

                                      SalePage_Title  休閒  防水/防潑水  有機棉  磨砂革  \
0                 (隱賣) Timberland 蠟質皮革保養保護劑|A1BSM000   0       0    0    0   
1    (隱賣)Timberland EK by Raeburn 深灰色口袋短袖T恤|A25NUM45   0       0    0    0   
2               (隱賣)Timberland 男款玫瑰粉色透氣長袖襯衫|A62XH662   0       0    0    0   
3                       Timberland 女款咖啡色皮帶 |A1D8L931   0       0    0    0   
4  (限定優惠)Timberland 女款小麥黃正絨面皮革 Lucia Way 靴 | A1T6...   0       0    0    0   

   logo  連帽  皮革  印花  ...  紫  粉  黑  小麥  灰  白  焦土  褐  米  鐵鏽  
0     0   0   1   0  ...  0  0  0   0  0  0   0  0  0   0  
1     0   0   0   0  ...  0  0  0   0  1  0   0  0  0   0  
2     0   0   0   0  ...  0  1  0   0  0  0   0  0  0   0  
3     0   0   0   0  ...  0  0  0   0  0  0   0  0  0   0  
4     0   0   1   0  ...  0  0  0   1  0  0 

In [86]:
# 創建特徵名稱-數字系統
name_dict = dict()
for i in range(2,118):
    name_dict[list(item.columns)[i]] = i-1
print(name_dict)

{'休閒': 1, '防水/防潑水': 2, '有機棉': 3, '磨砂革': 4, 'logo': 5, '連帽': 6, '皮革': 7, '印花': 8, '拼接': 9, '圓領': 10, '絨面': 11, '經典': 12, '口袋': 13, '全粒面': 14, '寬鬆': 15, '修身': 16, '運動': 17, '中筒': 18, '健行': 19, '刺繡': 20, '牛津': 21, '素面': 22, '戶外': 23, '撞色': 24, '卡其': 25, '工裝': 26, '標誌': 27, '彈性': 28, '胸前': 29, '迷彩': 30, '低筒': 31, '大樹': 32, '綁帶': 33, '絨面革': 34, '帆布': 35, '大學': 36, '保暖': 37, '條紋': 38, '牛仔': 39, '寬版': 40, '淺口': 41, '抓絨': 42, '緩震': 43, '針織': 44, '三合一': 45, '樹型': 46, '收納': 47, '彈力': 48, '聯名': 49, '環保': 50, '麂皮': 51, '棉質': 52, '透氣': 53, '贈品': 54, '輕量': 55, '高筒': 56, '織物': 57, '羽絨': 58, '尼龍': 59, '登山': 60, '舒適': 61, '慢跑': 62, '地球': 63, '斜紋': 64, '立領': 65, '直筒': 66, '長版': 67, '再生': 68, '短版': 69, '排汗': 70, '纖維': 71, '錐形': 72, '亞麻': 73, '網眼': 74, '時尚': 75, '多功能': 76, '織布': 77, '玫瑰': 78, '網布': 79, '圖騰': 80, '防風': 81, '輕質': 82, '窄版': 83, '燈芯絨': 84, '格子': 85, 'UV': 86, '套頭': 87, '簡約': 88, '抽繩': 89, '防異味': 90, '緊身': 91, '橡膠': 92, '開襟': 93, '刷毛': 94, '異材質': 95, '防撕': 96, '棉麻': 97, '羊毛': 98, '法蘭絨': 99, '卡

In [88]:
# 創建商品名稱-標籤系統
item_dict = dict()
for _, row in tqdm(item.iterrows()):
    features = [col for col, value in row.items() if value == 1]
    item_dict[row['SalePage_Id']] = features
print(item_dict)

16390it [00:00, 16890.85it/s]

{'jq7lb1I7BO5fTOdpb+5HIg==': ['皮革'], '/A2P26xd+Fdx5fqwlBHklQ==': ['口袋', '灰'], 'n86xSkA4ACr3tQy4qC/qHg==': ['透氣', '玫瑰', '粉'], 'XEeHbHVhPtbIyUn7/EO/JQ==': ['咖啡/棕'], 'i86M+aBbkl6WhQUOhXe4TQ==': ['皮革', '絨面', '黃', '小麥'], 'CFwy5MdcnF7U9S8+J9fyVw==': ['皮革', '絨面', '黃', '小麥'], '0P2P7Bc7vXGo4qIEtn7GyQ==': ['磨砂革', '黑'], 'SsomA8CSbLad1dSAbmz36A==': ['有機棉', '刺繡', '藍'], 'f0+ER3t2C21VY6f1MxHddg==': ['有機棉', '刺繡', '藍'], '6SX6DjqwnSSB85MNy2FYEA==': ['有機棉', '戶外', '白'], 'Fdoc3q0L90jaoD85jPom/Q==': ['有機棉', '戶外', '白'], 'IXMOoOrpPHCYlxDVaAYfGw==': ['有機棉', '戶外', '藍'], '8ck9lk1JC6pSuaOAYd/y+A==': ['拼接', '圓領', '撞色', '黃', '黑'], '/KVdI5kJ3SlQwJsx3xzcbA==': ['拼接', '圓領', '撞色', '黃', '黑'], '9Det2c8sjM+hGGF5MbKJ5A==': ['有機棉', '連帽', '拼接', '藍', '紫'], 'nXOy7nhMA7nbP90eNHTzug==': ['有機棉', '連帽', '拼接', '藍', '紫'], 'aExVCCSQV21Cpiu50Bym6Q==': ['連帽', '拼接', '撞色', '黃', '黑'], 'yFyvEsJCdij6QZY5Fy340A==': ['有機棉', '寬版', '粉'], '+2u0U2WYd+kUEVxqErRA7A==': ['有機棉', '寬版', '粉'], 'NqTiVpFuW8pQ+TLQhj0wBw==': ['燈芯絨', '紅'], 'EE2dtZcp+HeQaYtkuA

In [28]:
OrderSlaveData = pd.read_csv("D:/91APP_DataSet_2023/91APP_DataSet_2023/OrderSlaveData.csv").dropna()
OrderSlaveData.head()

C:\Users\user\AppData\Local\Temp\ipykernel_7872\1429134747.py:1: DtypeWarning: Columns (6,7,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  OrderSlaveData = pd.read_csv("D:/91APP_DataSet_2023/91APP_DataSet_2023/OrderSlaveData.csv").dropna()


,ShopMemberId,TradesGroupCode,TradesSlaveCode,OrderDateTime,OrderFinishDateTime,ChannelType,ChannelDetail,PaymentType,ShippingType,OuterProductSkuCode,ProductSkuCode,SalePageId,Qty,UnitPrice,SubtotalSalesAmount,SubtotalPrice,SubtotalPromotionDiscount,SubtotalCouponDiscount,SubtotalLoyaltyPointDiscount,StatusDef
0,j4z50WByupJrGQHeBCpkEts9DeTNF0UbqTq+yS9kJbs=,7Xl/dkMc7A6gW1HNXJGkzv5c0k+WnnpFoOnaUrfwHyo=,POlAG6iTdfU3Iwqx2urfsf2kVcuWTjyeCriR4bn8IGw=,2018-01-20 18:46:12.637,2018-01-24 20:46:38.700,OfficialECom,iOSApp,Family,Family,34VsVcDsYIMUssAvv+obZs0+EWZX5ybHuN5SY5a8tc4=,qPku+OahlP1erSYX2Adhmw==,Js+Ot+KsuHPoB5MspwTAXA==,1,1700.0,1700.0,1700.0,0.0,0.0,0.0,Finish
1,Vf3Tpdb5zhZXexeleFYlNvM3nMs73OwJ7aaRo252aGg=,uxIXgFJ9TgwSqQsI3ymwqEdXu1nabB8XE93vfIHEFTI=,rbOaXF0mDzm0JMb0GqRHXDFbPVO1Uttx4XwwtFXAfbE=,2022-03-14 11:03:06.780,2022-03-16 16:12:46.320,OfficialECom,iOSApp,CreditCardOnce,Home,v//VMh9dm83dMohLT7y02LDSWcQT4Yuuyfg9oFe2hO0=,MKkg38wgvpb3IgGCeBstrg==,vdtjjCeCaLWFmN4fAMeCqg==,1,1700.0,775.0,1700.0,-425.0,-500.0,0.0,Finish
2,MrXD+ZpCs4EqPPu+zk5/yuR2HYHoO/2xOCGrM5msGlE=,UoONBGB/2QCsORuVxVx1LlNtdQdJAx1ziz3yPPbVWpI=,Ev7hPM3iKpQQ4geRzEnlHc9NXehcVe+TD4/Ptu8h67w=,2022-02-13 07:44:29.637,2022-02-14 09:21:05.287,OfficialECom,iOSApp,CreditCardOnce,Home,f2NoanWkC4x07eEBlQr9fTt3K9ZZyEoRlMLTMGamfEQ=,Zfy7zkhJUhnYoOabdX6vTQ==,S+RYOEOGPfqfaO+FWpMMxg==,1,3400.0,3400.0,3400.0,0.0,0.0,0.0,Finish
3,t8CKFyQhD3jYfCRrG6K1vU5Q5xyobtJGPNdFrUFqGYU=,k0agP5th/G6b8v9jXhDaur6V9atcJ4zXx39Q+RU5zzA=,SeG7CmlMDlExCO7s1in1ZhskRCQ9j+YrkaXYzvMV5IE=,2021-05-19 18:06:26.327,2021-05-20 09:12:01.573,OfficialECom,iOSApp,CreditCardOnce,Home,ingtzUYdZCQ/SGleZOzs2r2ZH9O3yZt5I33VDT4/TEo=,Dh7QiIBF32PpELbzEARlvg==,9gh7IvY0PPULf1XRi3VTYQ==,1,1190.0,1190.0,1190.0,0.0,0.0,0.0,Finish
4,rxpBkm3xQryMLO9erBk/IO/suuJMvxFle79DohbloQI=,OpC5hcoN0rWKzvL1hUmZO/YdbeXd6D4VfsSvre9Gvy0=,jI0eoQVSaCdH0topQ+pL3ZDUUic4f0lvMZnTaVlFE9o=,2018-05-20 22:06:51.273,2018-05-24 04:45:22.820,OfficialECom,iOSApp,SevenEleven,SevenEleven,xps9V0u8l+ymqKvWQjr1yUXjOJiLOKxY4rH1+4V8N0U=,f+K4jd1DFpzaHu+HGkCnzg==,Nf+fQZjn3lhYBfu/PYEuSA==,1,1510.0,990.0,1510.0,-520.0,0.0,0.0,Finish


In [31]:
# filtered_df = OrderSlaveData[OrderSlaveData['OrderDateTime'] == 2021]
OrderSlaveData['OrderDateTime'] = pd.to_datetime(OrderSlaveData['OrderDateTime'])
OrderSlaveData2 = OrderSlaveData[OrderSlaveData['OrderDateTime'].dt.year == 2022]  #560萬筆之中只有13萬筆是2022年
OrderSlaveData2['OrderDateTime']


1         2022-03-14 11:03:06.780
2         2022-02-13 07:44:29.637
5         2022-01-04 21:18:09.880
12        2022-11-11 08:42:40.613
18        2022-03-25 15:26:27.070
                    ...          
5637685   2022-05-06 15:16:36.070
5637692   2022-12-27 00:22:53.773
5637693   2022-05-09 23:48:03.630
5637714   2022-05-15 00:18:18.990
5638235   2022-09-23 08:01:03.873
Name: OrderDateTime, Length: 138753, dtype: datetime64[ns]

In [89]:
#Case 1:透過購買紀錄來進行預測推薦
member_id = input("請輸入使用者編號：")
buylist = []
for _, row in tqdm(OrderSlaveData2.iterrows()):
    if row["ShopMemberId"] == member_id:
        buylist.append(item_dict[row["SalePageId"]])
print(buylist)

請輸入使用者編號： MrXD+ZpCs4EqPPu+zk5/yuR2HYHoO/2xOCGrM5msGlE=


138753it [00:05, 27445.35it/s]

[['logo', '胸前', '燈芯絨', '白'], ['logo', '胸前', '燈芯絨', '白'], ['多功能', '藍'], ['有機棉', '工裝', '抽繩', '粉', '灰']]


In [90]:
# 將購買紀錄轉換成數字，並且計算購買項目中出現過的數量
d = [item for sublist in buylist for item in sublist]
num_buylist = []
for i in d:
    num_buylist.append(name_dict[i])
num_buylist = sorted(num_buylist)
set_buylist = sorted(set(num_buylist))
from collections import Counter 
print(num_buylist)
print(Counter(num_buylist))

[3, 5, 5, 26, 29, 29, 76, 84, 84, 89, 106, 108, 111, 112, 112]
Counter({5: 2, 29: 2, 84: 2, 112: 2, 3: 1, 26: 1, 76: 1, 89: 1, 106: 1, 108: 1, 111: 1})


In [91]:
# 呼叫購物人資料
MemberData = pd.read_csv("D:/91APP_DataSet_2023/91APP_DataSet_2023/MemberData.csv")
MemberData.head()

C:\Users\user\AppData\Local\Temp\ipykernel_7872\2895218038.py:2: DtypeWarning: Columns (3,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  MemberData = pd.read_csv("D:/91APP_DataSet_2023/91APP_DataSet_2023/MemberData.csv")


,ShopMemberId,RegisterSourceTypeDef,RegisterDateTime,Gender,Birthday,APPRefereeId,APPRefereeLocationId,IsAppInstalled,IsEnableEmail,IsEnablePushNotification,IsEnableShortMessage,FirstAppOpenDateTime,LastAppOpenDateTime,MemberCardLevel,CountryAliasCode
0,HUDKwsme4lfsXH78PxhdsWp4FSonEjSJDliWqbmY5wQ=,LocationWizard,2019-11-09 17:27:42.783,Male,1970-01-01,NaN,NaN,True,True,True,True,2019-11-09 17:40:27.383,2023-03-29 16:58:40.490,20,TW
1,i3+s315OzIdaCQ+Nb1Uzvp7HVS4dTtSfUsQY7U0wzvA=,LocationWizard,2020-10-04 16:06:10.080,Male,1970-01-01,aAUUnpRdvBgr74w8+zHLiA==,r/ngQhN6tZP3bXeSXj/e0A==,True,True,True,True,2020-10-06 20:23:21.690,2022-02-05 19:12:41.807,10,TW
2,yDTBj6LrVHsUSCLZcMD23u628nb+ksFHmmMn1mAYq5Y=,LocationWizard,2020-08-16 18:47:28.817,Male,1970-01-01,ZRoLZiNWqVRRtWrC8vKmyQ==,TqfYonU/dikRdQ4pfXe+Vg==,True,True,True,True,2020-08-17 08:58:36.417,2022-07-29 11:30:53.870,10,TW
3,2t8mHJt0pp0UCAkJ9RxTxxTuca7QMJ0GJ9gio7pmuYA=,AndroidApp,2019-12-05 08:46:48.063,Male,1970-01-01,NaN,NaN,True,True,True,True,2019-12-04 09:41:30.833,2020-05-14 07:55:01.620,10,TW
4,acrCG5ZNR0f5y5ixoOx+07J6a47QXfUvmowKJJDas34=,Store,2013-12-21 00:00:00.000,Male,1970-01-01,NaN,NaN,True,True,True,True,2017-01-05 00:26:29.823,2018-12-28 20:54:25.117,10,TW


In [92]:
#決定他要使用哪三個籃子
row2 = MemberData.loc[MemberData['ShopMemberId'] == member_id]
age = row2["Birthday"].values
age = 2023-int(age[0][0:4])
level = int(int(row2["MemberCardLevel"])/10)
print(age)
print(level)

if age <= 20:
    agetype = "age1"
elif age <= 30:
    agetype = "age2"
elif age <= 40:
    agetype = "age3"
elif age <= 50:
    agetype = "age4"
else:
    agetype = "age5"
leveltype = "level"+str(level)

print(agetype+leveltype)

output_dict = dict()

42
2
age4level2


In [93]:
# 計算規則和每個的score-age_based(出現次數、同時出現次數，總體乘數：1)
appear = []
age_rule = all_rules[agetype]
# print(age_rule.head())
# print(set_buylist)
for _, row in age_rule.iterrows():
    if set(row["buy"]).issubset(set_buylist):
        for i in row["maybuy"]:
            multiplier = 1/len(row["buy"])
            if i not in appear:
                appear.append(i)
                output_dict[i] = row["confidence"]*row["lift"]*multiplier
            else:
                output_dict[i] += row["confidence"]*row["lift"]*multiplier
            for j in row["buy"]:
                output_dict[i] = output_dict[i] * Counter(num_buylist)[j]
            
print(output_dict)

{1: 0.4834714655819563, 5: 0.9929123793597181, 3: 3.6203094526734487, 8: 0.8467734609590388, 106: 0.6009148898290465}


In [94]:
# 計算規則和每個的score-level_based(出現次數、同時出現次數，總體乘數：1)
appear = []
level_rule = all_rules[leveltype]
for _, row in level_rule.iterrows():
    if set(row["buy"]).issubset(set_buylist):
        for i in row["maybuy"]:
            multiplier = 1/len(row["buy"])
            if i not in appear:
                appear.append(i)
                output_dict[i] = row["confidence"]*row["lift"]*multiplier
            else:
                output_dict[i] += row["confidence"]*row["lift"]*multiplier
            for j in row["buy"]:
                output_dict[i] = output_dict[i] * Counter(num_buylist)[j]
            
print(output_dict)

{1: 0.4834714655819563, 5: 0.8082049769231238, 3: 7.764914224396007, 8: 0.6762577680350652, 106: 0.5552496443829187, 112: 0.407293704258922}


In [95]:
# 計算規則和每個的score-age+level_based(出現次數、同時出現次數，總體乘數：3)
appear = []
age_level_rule = all_rules[agetype+leveltype]
for _, row in age_level_rule.iterrows():
    if set(row["buy"]).issubset(set_buylist):
        for i in row["maybuy"]:
            multiplier = 1/len(row["buy"])
            if i not in appear:
                appear.append(i)
                output_dict[i] = row["confidence"]*row["lift"]*multiplier*3
            else:
                output_dict[i] += row["confidence"]*row["lift"]*multiplier*3
            for j in row["buy"]:
                output_dict[i] = output_dict[i] * Counter(num_buylist)[j]
            
sorted_dict = dict(sorted(output_dict.items(), key=lambda x: x[1], reverse=True))
print(sorted_dict)

{3: 23.001832808969368, 5: 2.338694418164617, 8: 1.6775167968732845, 106: 1.6230058463249941, 1: 0.4834714655819563, 112: 0.407293704258922}


In [96]:
#結果：無取LOG版本
name_dict_reverse = dict()
import math
for i in range(2,118):
    name_dict_reverse[i-1] = list(item.columns)[i]
for i, j in sorted_dict.items():
    print(name_dict_reverse[i], j)

有機棉 23.001832808969368
logo 2.338694418164617
印花 1.6775167968732845
藍 1.6230058463249941
休閒 0.4834714655819563
白 0.407293704258922


In [97]:
# 結果：LOG版本(大樣本降低權重)
for i, j in sorted_dict.items():
    sorted_dict[i] =  sorted_dict[i]/math.log(item[name_dict_reverse[i]].sum(),10)
sorted_dict2 = dict(sorted(sorted_dict.items(), key=lambda x: x[1], reverse=True))
for i, j in sorted_dict2.items():
    print(name_dict_reverse[i], j)

有機棉 6.979628122040438
logo 0.7405476242429689
印花 0.5413713047733338
藍 0.46595798486537454
休閒 0.14309134043387547
白 0.12789110439951112


In [98]:
print(Counter(d))

Counter({'logo': 2, '胸前': 2, '燈芯絨': 2, '白': 2, '多功能': 1, '藍': 1, '有機棉': 1, '工裝': 1, '抽繩': 1, '粉': 1, '灰': 1})


In [ ]:
"""
已經顯示的需求：
有機棉 1
磨砂革 3
休閒 1
防水/防潑水 4
健行 4
藍 4
印花 3

隱性需求：
logo
皮革 

可以多琢磨的需求：
有機棉：6.7-1=5.7
磨砂革：3.6-3=0.6
休閒 1.5-1=0.5
新出現：
logo 0.56*5=2.8
皮革 0.74*5=3.7

最後推薦名單：
有機棉、皮革、logo、摩砂革、休閒

"""

In [23]:
#test
from apyori import apriori
frequent_itemsets = apriori(all_basket[0], min_support=0.03,min_confidence=0.2, min_lift=1.5, use_colnames=False)
rules = list(frequent_itemsets)
print(len(all_basket[0]))
columns = ['buy', 'maybuy', 'support', 'confidence', 'lift']
df = pd.DataFrame(columns=columns)
for product in rules:
    for j in range(len(product[2])):
        data = [list(product[2][j][0]), list(product[2][j][1]), float(product[1]),float(product[2][j][2]),float(product[2][j][3])]
        df = pd.concat([df, pd.DataFrame([data], columns=df.columns)], ignore_index=True)
print(df)

211
           buy    maybuy   support confidence      lift
0          [1]     [111]  0.047393   0.222222  2.131313
1        [111]       [1]  0.047393   0.454545  2.131313
2          [2]       [4]  0.094787   0.526316  2.019139
3          [4]       [2]  0.094787   0.363636  2.019139
4          [2]       [7]  0.037915   0.210526  2.613003
5          [7]       [2]  0.037915   0.470588  2.613003
6          [2]      [11]  0.037915   0.210526  2.776316
7         [11]       [2]  0.037915        0.5  2.776316
8        [104]       [2]  0.033175   0.466667  2.591228
9          [2]     [110]  0.061611   0.342105  2.489111
10       [110]       [2]  0.061611   0.448276  2.489111
11         [3]       [5]  0.056872      0.375  3.596591
12         [5]       [3]  0.056872   0.545455  3.596591
13         [3]       [8]  0.042654    0.28125  3.296875
14         [8]       [3]  0.042654        0.5  3.296875
15         [3]     [112]  0.066351     0.4375  2.977823
16       [112]       [3]  0.066351   0.45161

In [61]:
"""apriori with bugs(not shown)"""

# from mlxtend.preprocessing import TransactionEncoder
# from mlxtend.frequent_patterns import apriori, association_rules
# for basket in all_basket:
#     te = TransactionEncoder()
#     te_ary = te.fit_transform(basket)
#     df = pd.DataFrame(te_ary, columns=te.columns_)
#     frequent_itemsets = apriori(df, min_support=0.03, use_colnames=False)
#     rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
#     print("頻繁項目集:")
#     ff = 0
#     for i in basket:
#         if 110 in i:
#             ff += 1
#     print(ff)
#     print(ff/len(basket))
#     print(frequent_itemsets)
#     # print("\n關聯規則:")
#     # print(rules)
#     

頻繁項目集:
29
0.13744075829383887
     support     itemsets
0   0.213270          (0)
1   0.180095          (1)
2   0.151659          (2)
3   0.260664          (3)
4   0.104265          (4)
5   0.066351          (5)
6   0.080569          (6)
7   0.085308          (7)
8   0.061611          (8)
9   0.033175          (9)
10  0.075829         (10)
11  0.047393         (11)
12  0.061611         (13)
13  0.042654         (14)
14  0.033175         (18)
15  0.075829         (20)
16  0.037915         (31)
17  0.056872         (67)
18  0.071090         (68)
19  0.052133         (69)
20  0.071090         (70)
21  0.303318         (72)
22  0.137441         (73)
23  0.104265         (74)
24  0.146919         (75)
25  0.056872         (77)
26  0.071090       (0, 3)
27  0.071090      (0, 72)
28  0.047393      (0, 74)
29  0.094787       (1, 3)
30  0.037915       (1, 6)
31  0.037915      (1, 10)
32  0.033175      (1, 68)
33  0.052133      (72, 1)
34  0.061611      (73, 1)
35  0.056872       (2, 4)
36  0.04